In [1]:
import pandas as pd
import numpy as np
import datetime
from scrape import scrape

## Table of Contents

1. [Scrape Data](#1.-Scrape-Data)
2. [Data Cleaning](#2.-Data-Cleaning)
3. [Feature Engineering](#3.-Feature-Engineering)
4. [Reshape Data for Tableau](#4.-Reshape-Data-for-Tableau)
5. [Save Final Dataframe](#5.-Save-Final-Dataframe)

## 1. Scrape Data

In [2]:
# scrape lens data from nikon.com
# http://www.nikonusa.com/en/nikon-products/camera-lenses/all-lenses/index.page

lenses = scrape()

# filter columns
keep = ['Approx. Dimensions (Diameter x Length)',
        'Approx. Weight',
        'Autofocus',
        'Focal Length',
        'Focal Length Range',
        'Format',
        'Lens Elements',
        'Lens Groups',
        'Maximum Aperture',
        'Minimum Aperture',
        'Price',
        'VR (Vibration Reduction) Image Stabilization',
        'Url']

lenses = lenses[keep]

# Rename Columns
lenses.rename(index = str,
              inplace = True,
              columns = {'Approx. Dimensions (Diameter x Length)': 'Dimensions',
                         'Approx. Weight': 'Weight',
                         'VR (Vibration Reduction) Image Stabilization':'VR'})

# lenses.head()

## 2. Data Cleaning

In [3]:
# Remove rows that don't contain Focal Length data
lenses = lenses[~(lenses['Focal Length'].isnull() & lenses['Focal Length Range'].isnull())]

# Check for nulls and view data types

#lenses.isnull().any()
lenses[['Lens Elements','Lens Groups']] = lenses[['Lens Elements','Lens Groups']].apply(pd.to_numeric)

# lenses.info()
# lenses.head()

### Dimensions

In [4]:
# Calculate length of dimension data list. Should contain 4 values
# Format: diameter_in (diameter_mm) x length_in (length_mm)

lenses['dim_test'] = lenses.Dimensions.apply(lambda x: x.split(',')).str.len()

# Find incorrect values and correct them
lenses.iloc[23,lenses.columns.get_loc('Dimensions')] = '3.3,83,3.7,95'
lenses.iloc[27,lenses.columns.get_loc('Dimensions')] = '2.6,66,1.7,43'

# Verify Fix
# lenses['dim_test'] = lenses.Dimensions.apply(lambda x: x.split(',')).str.len()
#lenses[lenses['dim_test'] != 4]

In [5]:
# Keep only 1st and 3rd values (diameter_in,length_in)
lenses['Diameter (In.)'] = lenses['Dimensions'].apply(lambda x: float(x.split(',')[0]))
lenses['Length (In.)'] = lenses['Dimensions'].apply(lambda x: float(x.split(',')[2]))

lenses.drop(['dim_test','Dimensions'],inplace=True,axis=1)
#lenses.sample(5)

### Weight

In [6]:
lenses['wt_test'] = lenses.Weight.apply(lambda x: x.split(',')).str.len()

lenses.iloc[82,lenses.columns.get_loc('Weight')] = '21.9'

lenses['Weight (oz.)'] = lenses['Weight'].apply(lambda x: float(x.split(',')[0]))

lenses.drop(['wt_test','Weight'],inplace=True,axis=1)

### Autofocus

In [7]:
# Set Null values to "No"
lenses.loc[lenses['Autofocus'].isnull(),'Autofocus'] = 'No'

### Focal Length

In [8]:
# 'Focal Length' 'Focal Length Range'
# get min and max focal length values

# Combine non-Null values in 'Focal Length' and 'Focal Length Range'
lenses['lengths'] = lenses['Focal Length'].fillna(lenses['Focal Length Range'])

def focalLengths(s):
    # assign min and max focal length variables
    s = [float(ss) for ss in s.split(',')]
    
    if len(s) == 1:
        minFlen,maxFlen = s[0],s[0]
    else:
        minFlen,maxFlen = s
    return minFlen,maxFlen

lenses[['Min. Focal Length (mm)','Max. Focal Length (mm)']] = lenses['lengths'].apply(focalLengths).apply(pd.Series)
lenses.drop(['lengths'],inplace=True,axis=1)
# lenses.head(2)

### Format

In [9]:
# Assign FX or DX values
lenses.loc[lenses.Format == 'FX/35mm,DX','Format'] = 'FX'
lenses.loc[lenses.Format == 'FX/35mm','Format'] = 'FX'

# lenses.Format.value_counts()

### Aperture

In [10]:
def apertureBounds(s):
    # Assign upper and lower bounds for Aperture f-numbers
    s = [float(ss) for ss in s.split('-')]
    
    if len(s) == 1:
        minFnum,maxFnum = s[0],s[0]
    else:
        minFnum = min(s)
        maxFnum = max(s)
    return minFnum,maxFnum

# Upper and Lower Bounds for Maximum Aperture
max_aperture_cols = ['Max. Aperture (Upper Bound)','Max. Aperture (Lower Bound)']
lenses[max_aperture_cols] = lenses['Maximum Aperture'].apply(apertureBounds).apply(pd.Series)

# Upper and Lower Bounds for Minimum Aperture
min_aperture_cols = ['Min. Aperture (Upper Bound)','Min. Aperture (Lower Bound)']
lenses[min_aperture_cols] = lenses['Minimum Aperture'].apply(apertureBounds).apply(pd.Series)

### Vibration Reduction

In [11]:
# Set Null values to "No"
lenses.loc[lenses['VR'].isnull(),'VR'] = 'No'

# lenses['VR'].value_counts()

## 3. Feature Engineering

In [12]:
#  # Assign name to each lens

def lensName(s):
    # Concatenate focal length range with aperture range
    name = ''

    if pd.isnull(s[0]):
        # telephoto type lens
        aps = s[1].split(',')
        
        fMin = aps[0]
        fMax = aps[1]
        
        name += aps[0] + '-' + aps[1] + 'mm'
    else:
        # prime lens
        name += s[0] + 'mm'
    
    return name + ' f' + s[2]
    
    
lenses['name'] = lenses[['Focal Length','Focal Length Range','Maximum Aperture']].apply(lensName,axis=1)

In [13]:
# Define lens type (Prime vs. Zoom)
def lensType(s):
    # Return "Prime" lens type if Focal Length Range is Null
    if pd.isnull(s):
        return 'Prime'
    return 'Zoom'
    
lenses['Lens Type'] = lenses['Focal Length Range'].apply(lensType)

In [14]:
# Focal Length Bins (Wide Angle, Standard, Telephoto, etc.)
lenses['Focal Length Group'] = pd.cut(lenses['Max. Focal Length (mm)'],
                                      bins=[0, 24, 35, 85, 135, 300, 1000],
                                      include_lowest=True,
                                      right=False,
                                      labels=['Ultra Wide Angle',
                                              'Wide Angle',
                                              'Standard',
                                              'Short Telephoto',
                                              'Medium Telephoto',
                                              'Super Telephoto'])

# Drop Unecessary Columns
drop_cols = ['Focal Length',
             'Focal Length Range',
             'Maximum Aperture',
             'Minimum Aperture']

lenses.drop(drop_cols,inplace=True,axis=1)

#lenses.head()

## 4. Reshape Data for Tableau

In [15]:
# Empty df to preserve index
empty = lenses[['Url']]

# Stacked focal lengths for tableau plotting
flens = pd.melt(lenses,
                id_vars = ['Url'],
                col_level = 0,
                value_name = 'Focal Length',
                value_vars = ['Min. Focal Length (mm)',
                              'Max. Focal Length (mm)'])

flens = empty.reset_index().merge(flens, on = 'Url')
flens = flens.sort_values(['index','Focal Length'],ascending=[True,False])

# Stacked max. apertures for tableau plotting
aperts = pd.melt(lenses,
                id_vars = ['Url'],
                col_level = 0,
                value_name = 'Max. Aperture',
                value_vars = ['Max. Aperture (Upper Bound)',
                              'Max. Aperture (Lower Bound)'])

aperts = empty.reset_index().merge(aperts, on = 'Url')
aperts = aperts.sort_values(['index','Max. Aperture'],ascending=[True,False])

## 5. Save Final Dataframe

In [16]:
measures = pd.concat([aperts[['index','Max. Aperture']],flens['Focal Length']],axis = 1)

final = lenses.merge(measures, left_index = True, right_on='index').set_index('index')

today = datetime.datetime.today().strftime('%Y%m%d')

final.to_csv('nikon_lenses_' + today + '.csv')